In [ ]:
import numpy as np
# Перед началом работы с библиотеками pandas и seaborn необходимо установить их в Python.
# Сделать это можно, исполнив данную ячейку
!pip3 install pandas
!pip3 install seaborn

In [62]:
import math
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Настройка, которая позволяет сделать графики, которые рисует seaborn, более крупными
sns.set(rc={'figure.figsize': (11.7, 8.27)})

In [ ]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

## Среднее значение в колонке csv-файла

In [ ]:
def split_line(line, sep):
    res = []

    prev_ind = -1
    in_quotes = False

    for i in range(len(line)):
        ch = line[i]

        if ch == '"':
            in_quotes = not in_quotes

        if not in_quotes and ch == sep:
            res.append(line[prev_ind + 1:i].strip('"'))
            prev_ind = i

    if prev_ind < len(line) - 1:
        res.append(line[prev_ind + 1:len(line)].strip('"'))
    elif prev_ind == len(line) - 1:
        res.append('')

    return res
    
    res = []
    
    prev_ind = -1
    in_quotes = False
    
    for i in range(len(line)):
        ch = line[i]
        
        if ch == '"':
            in_quotes = not in_quotes
            
        if not in_quotes and ch == sep:
            res.append(line[prev_ind + 1:i].strip('"'))
            prev_ind = i
            
    if prev_ind < len(line) - 1:
        res.append(line[prev_ind + 1:len(line)].strip('"'))
    elif prev_ind == len(line) - 1:
        res.append('')
    
    return res

In [ ]:
def read_csv_and_calc_mean_solution(lines, sep, column_name):
    """
    Принимает список строк из csv-файла и возвращает среднее значение в заданной колонке.
    В первой строке csv-файла всегда представлены названия его колонок.
    
    Аргументы:
        lines: Список строк, прочитанных из csv-файла.
        sep: Разделитель, который используется в строках для отделения различных значений.
        column_name: Название колонки с целочисленными значениями, по которой нужно посчитать среднее.
        
    Возвращаемое значение:
        Среднее значение в заданной колонке, округлённое до 2 знаков
        после запятой с помощью функции round_to_2.
    """
    res = [0]*len(lines)
    for k in range(len(lines)):
        res[k] = split_line(lines[k], sep)

    i = 0
    sum = 0
    while res[0][i] != column_name:
        i += 1

    for j in range(1, len(lines)):
        sum += int(res[j][i])

    return round(sum/(len(lines) - 1), 2)

    pass

In [ ]:
def read_csv_and_calc_mean_tests():
    example_1_lines = [
        'Жанр,"Продолжительность, часы","Продолжительность, минуты",Страна производства,Год производства',
        'Боевик,0,36,Италия,1996',
        'Комедия,1,23,Россия,1999',
        'Триллер,2,21,США,2006',
    ]
    example_1_sep = ','
    example_1_column = 'Год производства'
    example_1_res = 2000.33
    
    assert read_csv_and_calc_mean_solution(example_1_lines, example_1_sep, example_1_column) == example_1_res
    
    example_2_lines = [
        'Жанр,"Продолжительность, часы","Продолжительность, минуты",Страна производства,Год производства',
        'Боевик,0,36,Италия,1996',
        'Комедия,1,23,Россия,1999',
        'Триллер,2,21,США,2006',
        'Комедия,1,48,США-Россия,2012',
        'Драма,2,3,Франция,2016',
        'Документальный,2,47,Германия,2001'
    ]
    example_2_sep = ','
    example_2_column = 'Продолжительность, минуты'
    example_2_res = 29.67
    
    assert read_csv_and_calc_mean_solution(example_2_lines, example_2_sep, example_2_column) == example_2_res
    
    print('Все тесты прошли успешно!')

In [ ]:
read_csv_and_calc_mean_tests()

## Фильтрация таблиц

In [ ]:
# В переменную movies загружена таблица из файла movies_data.csv
movies = pd.read_csv('movies_data.csv', sep=',')

# Удаление фильмов, у которых не указан год производства
movies = movies[~movies['Год производства'].isna()]

## Комбинирование колонок и расчёт статистик

In [ ]:
# В переменную movies_budget загружена таблица из файла movies_budget.csv
movies_budget = pd.read_csv('movies_budget.csv', sep=';')

## Агрегированные статистики

In [ ]:
# В переменную movies_budget загружена таблица из файла movies_budget.csv
movies_budget = pd.read_csv('movies_budget.csv', sep=';')

In [ ]:
def transform_countries_info(movies_data):
    """
    Принимает таблицу с колонкой «Страна производства» и добавляет к таблице 5 новых колонок:
    по одной на каждую отдельную страну. Значение в каждой из колонок соответствует тому,
    принимала ли конкретная страна участие в производстве фильма.
    
    Аргументы:
        movies_data: Исходная таблица с клонкой «Страна производства».
        
    Возвращаемое значение:
        Нет. Функция модифицирует переданную таблицу, но ничего не возвращает.
    """
    
    countries = ['Россия', 'Германия', 'США', 'Италия', 'Франция']
    new_columns = [[] for i in range(len(countries))]
    
    for _, row in movies_data.iterrows():
        country_of_origin_united = row['Страна производства']
        countries_of_origin = country_of_origin_united.split('-')
        
        for i in range(len(countries)):
            country = countries[i]
            
            if country in countries_of_origin:
                new_columns[i].append('Да')
            else:
                new_columns[i].append('Нет')
                
    for i in range(len(countries)):
        movies_data[countries[i]] = new_columns[i]

## Визуальный анализ данных

In [ ]:
# В переменную many_factors загружена таблица из файла many_factors_data.csv
many_factors = pd.read_csv('many_factors_data.csv')

In [3]:
# В переменную uniform_and_not загружена таблица из файла uniform_and_not_data
uniform_and_not = pd.read_csv('uniform_and_not_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'uniform_and_not_data.csv'

In [85]:
movies_data = pd.read_csv('movies_data.csv')

In [86]:
data = movies_data[((movies_data['Страна производства'] == 'Россия') | ((movies_data['Жанр фильма'] == 'Триллер') & (movies_data['Год производства'] < 2000)))]

exp = movies_data[((movies_data['Страна производства'] == 'Россия') & ((movies_data['Жанр фильма'] == 'Триллер') & (movies_data['Год производства'] < 2000)))]

#data
len(data)


3637

In [58]:
#Задание с movies_budget.csv

movies_budget = pd.read_csv('movies_budget.csv', sep=';')



In [66]:
#1
cost = movies_budget['Стоимость билета']
count = movies_budget['Число зрителей']
#test = cost * count

ans = np.mean(cost * count)

ans

67278893.76

In [74]:
#2
budget = movies_budget['Бюджет']
profit = ans/2 - 1.5 * budget

max(profit)

33008851.380000003

In [82]:
#3

comedy = movies_budget[(movies_budget['Жанр фильма'] == 'Триллер')]
comedy = comedy['Число зрителей']

sum(comedy)

30519114

In [83]:
# Заданный словарь
genres_dict = {
    'Комедия': 9920914,
    'Драма': 144536483,
    'Документальный': 10093471,
    'Боевик': 20859969,
    'Триллер': 30519114
}

# Сортировка словаря по значениям
sorted_genres = sorted(genres_dict, key=lambda x: genres_dict[x])

# Вывод отсортированного словаря
for genre in sorted_genres:
    print(f'{genre} - {genres_dict[genre]}')


Комедия - 9920914
Документальный - 10093471
Боевик - 20859969
Триллер - 30519114
Драма - 144536483
